<a href="https://colab.research.google.com/github/sakamototaisei/python_pf_colabo/blob/main/PF_%E6%A9%9F%E8%83%BD%E6%A0%AA%E4%BE%A1%E3%83%86%E3%82%AF%E3%83%8B%E3%82%AB%E3%83%AB%E8%A1%A8%E7%A4%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **株価のローソク足データを取得する**

In [ ]:
pip install yahoo_finance_api2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for yahoo_finance_api2: filename=yahoo_finance_api2-0.0.12-py3-none-any.whl size=3733 sha256=1b154c7edaa7481228e860342396c97daeeab98db2ba116b266a31fecbd4a50b
  Stored in directory: /root/.cache/pip/wheels/60/31/49/4f2dc65be0bb9c065b8f90701802babc434ad029dfaa9202df
Successfully built yahoo_finance_api2


In [ ]:
import sys
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError
import pandas as pd

In [ ]:
# SoftBank株価取得
my_share = share.Share('9984.T')
symbol_data = None
try:
    symbol_data = my_share.get_historical(
    share.PERIOD_TYPE_YEAR, 5,
    share.FREQUENCY_TYPE_DAY, 1)
except YahooFinanceError as e:
    print(e.message)
    sys.exit(1)
df = pd.DataFrame(symbol_data)
df["datetime"] = pd.to_datetime(df.timestamp, unit="ms")

In [ ]:
df.head()

,timestamp,open,high,low,close,volume,datetime
0,1522108800000,4090.0,4135.0,4061.0,4126.5,11316400,2018-03-27
1,1522195200000,4003.0,4008.5,3916.0,3961.0,16677200,2018-03-28
2,1522281600000,3990.0,3993.0,3879.0,3903.0,12893000,2018-03-29
3,1522368000000,3959.5,4019.0,3950.0,3975.0,11425000,2018-03-30
4,1522627200000,3957.5,3972.0,3930.0,3931.5,7635600,2018-04-02


# **Plotlyでローソク足チャートと描く**

## インストール

In [ ]:
pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

## plotly.graph_objects.Candlestick

In [ ]:
fig = go.Figure(
    data=[go.Candlestick(x=df['datetime'], open=df['open'], high=df['high'], low=df['low'], close=df['close'], showlegend=False)]
)

# 下にあるバーを非表示にする場合
# fig.update(layout_xaxis_rangeslider_visible=False)

fig.show()

## タイトル・ラベルの追加、フォーマットの変更

In [ ]:
fig.update_layout(
    title={
        'text': 'SoftBank(9984)の日足チャート',
        'y':0.9, # 縦軸でほぼ一番上表示
        'x':0.5, # 横軸でちょうど真ん中表示
    },
    yaxis_title='株価',
    xaxis_title='Date'
)

# 日付のフォーマット変更
fig.update_xaxes(tickformat='%Y/%m/%d')

# 株価のフォーマット変更(カンマ区切り)
fig.update_yaxes(separatethousands=True)

# 下にあるバーを非表示にする場合
# fig.update(layout_xaxis_rangeslider_visible=False)

fig.show()

## 営業日のみを表示する

特定のデータを非表示にする：randebreaks

不要な日付を抽出

In [ ]:
#日付一覧を取得
d_all = pd.date_range(start=df['datetime'].iloc[0],end=df['datetime'].iloc[-1])
d_obs = [d.strftime("%Y-%m-%d") for d in df['datetime']]
d_breaks = [d for d in d_all.strftime("%Y-%m-%d").tolist() if not d in d_obs]

In [ ]:
d_breaks[:10]

['2018-03-31',
 '2018-04-01',
 '2018-04-07',
 '2018-04-08',
 '2018-04-14',
 '2018-04-15',
 '2018-04-21',
 '2018-04-22',
 '2018-04-28',
 '2018-04-29']

In [ ]:
fig.update_xaxes(rangebreaks=[dict(values=d_breaks)])
fig.show()

営業日のみを表示することができ、不自然な空白表示がなくなった

# **複数チャートを組み合わせる**

出来高や移動平均線、いろんなテクニカル指標を追加する

## plotly.subplot.make_subplotsで複数チャートを描く

基本的な使い方としては、subplotsでfigを定義し、fig.add_traceで各チャートにグラフを追加していく流れになる

## 出来高を追加する

make_subplotsで、rows（行）とcols（列）を指定し、shared_xaxes=Trueとしてx軸を共有するように設定します。

x_titleでxのラベル名を設定しています。

 

vertical_spacingはチャート間の隙間の大きさで0〜1で指定します。

この設定は好みですが、僕はだいたい0.05にすることが多いです。適宜調整してみてください。

 

row_widthではrow(行）の高さを指定します。x_titleはその名の通り、x軸名です。

y軸名は2つあるので、fig.update_yaxesを使ってそれぞれ指定します。

 

add_traceでrowとcolを指定することで、任意の場所にチャートを描くことができます。

1行目にローソク足チャート、2行目に出来高の棒グラフを描けばOKです。

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
 
# figを定義
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_width=[0.2, 0.7], x_title='Date')
 
# Candlestick 
fig.add_trace(
    go.Candlestick(x=df['datetime'], open=df['open'], high=df['high'], low=df['low'], close=df['close'], showlegend=False),
    row=1, col=1
)
 
# Volume
fig.add_trace(
    go.Bar(x=df['datetime'], y=df["volume"], showlegend=False),
    row=2, col=1
)
 
# Layout
fig.update_layout(
    title={
        'text': 'SoftBank(9984)の日足チャート',
        'y':0.9,
        'x':0.5,
    },
)

fig.update_xaxes(
    rangebreaks=[dict(values=d_breaks)], # 非営業日を非表示設定
    tickformat='%Y/%m/%d' # 日付のフォーマット変更
)
 
# ラベル名の設定とフォーマット変更（カンマ区切り）
fig.update_yaxes(separatethousands=True, title_text='株価', row=1, col=1) 
fig.update_yaxes(title_text='出来高', row=2, col=1)
 
fig.update(layout_xaxis_rangeslider_visible=False) #追加

fig.show()

## 移動平均線を追加する

rolling()で移動平均線を計算

In [ ]:
# 移動平均線(SMA)を計算
df['SMA20'] = df['close'].rolling(window=20).mean()
df['SMA50'] = df['close'].rolling(window=50).mean()
df['SMA200'] = df['close'].rolling(window=200).mean()
df.tail()

,timestamp,open,high,low,close,volume,datetime,SMA20,SMA50,SMA200
1222,1679011200000,4876.0,4939.0,4852.0,4891.0,11392600,2023-03-17,5461.35,5799.74,5690.295
1223,1679270400000,4900.0,4965.0,4832.0,4851.0,10189600,2023-03-20,5415.70,5781.86,5687.845
1224,1679443200000,5000.0,5015.0,4912.0,4987.0,10272000,2023-03-22,5378.60,5765.28,5685.990
1225,1679529600000,4906.0,4975.0,4900.0,4960.0,8565500,2023-03-23,5341.60,5745.42,5683.895
1226,1679616000000,4950.0,4987.0,4910.0,4972.0,7941800,2023-03-24,5311.40,5725.48,5682.255


ローソク足チャートに移動平均線を加える

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
 
# figを定義
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_width=[0.2, 0.7], x_title='Date')
 
# Candlestick 
fig.add_trace(
    go.Candlestick(x=df['datetime'], open=df['open'], high=df['high'], low=df['low'], close=df['close'], showlegend=False),
    row=1, col=1
)

# 移動平均線
fig.add_trace(go.Scatter(x=df['datetime'], y=df["SMA20"], name="SMA20", mode="lines"), row=1, col=1)
fig.add_trace(go.Scatter(x=df['datetime'], y=df["SMA50"], name="SMA50", mode="lines"), row=1, col=1)
fig.add_trace(go.Scatter(x=df['datetime'], y=df["SMA200"], name="SMA200", mode="lines"), row=1, col=1)

# Volume
fig.add_trace(
    go.Bar(x=df['datetime'], y=df["volume"], showlegend=False),
    row=2, col=1
)
 
# Layout
fig.update_layout(
    title={
        'text': 'SoftBank(9984)の日足チャート',
        'y':0.9,
        'x':0.5,
    },
)

fig.update_xaxes(
    rangebreaks=[dict(values=d_breaks)], # 非営業日を非表示設定
    tickformat='%Y/%m/%d' # 日付のフォーマット変更
)
 
# ラベル名の設定とフォーマット変更（カンマ区切り）
fig.update_yaxes(separatethousands=True, title_text='株価', row=1, col=1) 
fig.update_yaxes(title_text='出来高', row=2, col=1)
 
fig.update(layout_xaxis_rangeslider_visible=False) #追加

fig.show()

## 複数のテクニカル指標を組み合わせる

・相対強度指数RSI

現在の株価が売られ過ぎか、買われ過ぎかを判断するときに使われるテクニカル指標の一つ。一定期間内の上げ下げ幅と下げ幅を合わせた全体の変動幅における上げ幅の割合を示す

・MACD(移動平均収束拡散手法)

短期の移動平均線と、中長期の移動平均線を使用することで、買いと売りを判断する手法です。ここで使用する移動平均線は通常の単純移動平均線SMAとは異なり、EMA新しい価格の方がより影響度が高という考えから、新しい価格の比重を高めて計算している

・移動平均線

ゴールデンクロスとデッドクロス

In [ ]:
def macd(df):
    FastEMA_period = 12  # 短期EMAの期間
    SlowEMA_period = 26  # 長期EMAの期間
    SignalSMA_period = 9  # SMAを取る期間
    df["MACD"] = df["close"].ewm(span=FastEMA_period).mean() - df["close"].ewm(span=SlowEMA_period).mean()
    df["Signal"] = df["MACD"].rolling(SignalSMA_period).mean()
    return df
 
 
def rsi(df):
    # 前日との差分を計算
    df_diff = df["close"].diff(1)
 
    # 計算用のDataFrameを定義
    df_up, df_down = df_diff.copy(), df_diff.copy()
    
    # df_upはマイナス値を0に変換
    # df_downはプラス値を0に変換して正負反転
    df_up[df_up < 0] = 0
    df_down[df_down > 0] = 0
    df_down = df_down * -1
    
    # 期間14でそれぞれの平均を算出
    df_up_sma14 = df_up.rolling(window=14, center=False).mean()
    df_down_sma14 = df_down.rolling(window=14, center=False).mean()
 
    # RSIを算出
    df["RSI"] = 100.0 * (df_up_sma14 / (df_up_sma14 + df_down_sma14))
 
    return df
 
# MACDを計算する
df = macd(df)
 
# RSIを算出
df = rsi(df)
 
df.tail()

,timestamp,open,high,low,close,volume,datetime,SMA20,SMA50,SMA200,MACD,Signal,RSI
1222,1679011200000,4876.0,4939.0,4852.0,4891.0,11392600,2023-03-17,5461.35,5799.74,5690.295,-235.077961,-146.664856,25.299145
1223,1679270400000,4900.0,4965.0,4832.0,4851.0,10189600,2023-03-20,5415.70,5781.86,5687.845,-250.368426,-162.868466,21.342513
1224,1679443200000,5000.0,5015.0,4912.0,4987.0,10272000,2023-03-22,5378.60,5765.28,5685.990,-248.645930,-179.929773,29.420849
1225,1679529600000,4906.0,4975.0,4900.0,4960.0,8565500,2023-03-23,5341.60,5745.42,5683.895,-246.616671,-197.805449,25.258141
1226,1679616000000,4950.0,4987.0,4910.0,4972.0,7941800,2023-03-24,5311.40,5725.48,5682.255,-241.259084,-212.844799,26.527331


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
 
# figを定義
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_width=[0.2, 0.2, 0.2, 0.7], x_title='Date')
 
# Candlestick 
fig.add_trace(
    go.Candlestick(x=df['datetime'], open=df['open'], high=df['high'], low=df['low'], close=df['close'], showlegend=False),
    row=1, col=1
)

# 移動平均線
fig.add_trace(go.Scatter(x=df['datetime'], y=df["SMA20"], name="SMA20", mode="lines"), row=1, col=1)
fig.add_trace(go.Scatter(x=df['datetime'], y=df["SMA50"], name="SMA50", mode="lines"), row=1, col=1)
fig.add_trace(go.Scatter(x=df['datetime'], y=df["SMA200"], name="SMA200", mode="lines"), row=1, col=1)

# MACD
fig.add_trace(go.Scatter(x=df['datetime'], y=df["MACD"], mode="lines", showlegend=False), row=3, col=1)
fig.add_trace(go.Scatter(x=df['datetime'], y=df["Signal"], mode="lines", showlegend=False), row=3, col=1)

# RSI
fig.add_trace(go.Scatter(x=df['datetime'], y=df["RSI"], mode="lines", showlegend=False), row=4, col=1)

# Volume
fig.add_trace(
    go.Bar(x=df['datetime'], y=df["volume"], showlegend=False),
    row=2, col=1
)
 
# Layout
fig.update_layout(
    title={
        'text': 'SoftBank(9984)の日足チャート',
        'y':0.9,
        'x':0.5,
    },
    height=700
)

fig.update_xaxes(
    rangebreaks=[dict(values=d_breaks)], # 非営業日を非表示設定
    tickformat='%Y/%m/%d' # 日付のフォーマット変更
)
 
# ラベル名の設定とフォーマット変更（カンマ区切り）
fig.update_yaxes(separatethousands=True, title_text='株価', row=1, col=1) 
fig.update_yaxes(title_text='出来高', row=2, col=1)
fig.update_yaxes(title_text="MACD", row=3, col=1)
fig.update_yaxes(title_text="RSI", row=4, col=1)
 
fig.update(layout_xaxis_rangeslider_visible=False) #追加

fig.show()

# **ここまでのまとめコード**

In [ ]:
pip install plotly

In [ ]:
pip install yahoo_finance_api2

In [ ]:
# 株価取得のインポート
import sys
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError
# 描画のインポート
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

In [ ]:
# 株価取得
my_share = share.Share('9449.T')
symbol_data = None
try:
    symbol_data = my_share.get_historical(
    share.PERIOD_TYPE_YEAR, 5,
    share.FREQUENCY_TYPE_DAY, 1)
except YahooFinanceError as e:
    print(e.message)
    sys.exit(1)
df = pd.DataFrame(symbol_data)
df["datetime"] = pd.to_datetime(df.timestamp, unit="ms")

# MACDとRSIの計算する関数
def macd(df):
    FastEMA_period = 12  # 短期EMAの期間
    SlowEMA_period = 26  # 長期EMAの期間
    SignalSMA_period = 9  # SMAを取る期間
    df["MACD"] = df["close"].ewm(span=FastEMA_period).mean() - df["close"].ewm(span=SlowEMA_period).mean()
    df["Signal"] = df["MACD"].rolling(SignalSMA_period).mean()
    return df

def rsi(df):
    # 前日との差分を計算
    df_diff = df["close"].diff(1)
 
    # 計算用のDataFrameを定義
    df_up, df_down = df_diff.copy(), df_diff.copy()
    
    # df_upはマイナス値を0に変換
    # df_downはプラス値を0に変換して正負反転
    df_up[df_up < 0] = 0
    df_down[df_down > 0] = 0
    df_down = df_down * -1
    
    # 期間14でそれぞれの平均を算出
    df_up_sma14 = df_up.rolling(window=14, center=False).mean()
    df_down_sma14 = df_down.rolling(window=14, center=False).mean()
 
    # RSIを算出
    df["RSI"] = 100.0 * (df_up_sma14 / (df_up_sma14 + df_down_sma14))
 
    return df
 
# MACDを計算する
df = macd(df)
 
# RSIを算出
df = rsi(df)


# 以下描画処理コード
# figを定義
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_width=[0.2, 0.2, 0.2, 0.7], x_title='Date')
 
# Candlestick 
fig.add_trace(
    go.Candlestick(x=df['datetime'], open=df['open'], high=df['high'], low=df['low'], close=df['close'], showlegend=False),
    row=1, col=1
)

# 移動平均線(SMA)を計算
df['SMA20'] = df['close'].rolling(window=20).mean()
df['SMA50'] = df['close'].rolling(window=50).mean()
df['SMA200'] = df['close'].rolling(window=200).mean()

# 移動平均線
fig.add_trace(go.Scatter(x=df['datetime'], y=df["SMA20"], name="SMA20", mode="lines"), row=1, col=1)
fig.add_trace(go.Scatter(x=df['datetime'], y=df["SMA50"], name="SMA50", mode="lines"), row=1, col=1)
fig.add_trace(go.Scatter(x=df['datetime'], y=df["SMA200"], name="SMA200", mode="lines"), row=1, col=1)

# MACD
fig.add_trace(go.Scatter(x=df['datetime'] , y=df["MACD"], mode="lines", showlegend=False), row=3, col=1)
fig.add_trace(go.Scatter(x=df['datetime'] , y=df["Signal"], mode="lines", showlegend=False), row=3, col=1)

# RSI
fig.add_trace(go.Scatter(x=df['datetime'] , y=df["RSI"], mode="lines", showlegend=False), row=4, col=1)

# Volume
fig.add_trace(
    go.Bar(x=df['datetime'], y=df["volume"], showlegend=False),
    row=2, col=1
)
 
# Layout
fig.update_layout(
    title={
        'text': '銘柄名(xxxx)の日足チャート',
        'y':0.9,
        'x':0.5,
    },
    height=700
)

fig.update_xaxes(
    rangebreaks=[dict(values=d_breaks)], # 非営業日を非表示設定
    tickformat='%Y/%m/%d' # 日付のフォーマット変更
)
 
# ラベル名の設定とフォーマット変更（カンマ区切り）
fig.update_yaxes(separatethousands=True, title_text='株価', row=1, col=1) 
fig.update_yaxes(title_text='出来高', row=2, col=1)
fig.update_yaxes(title_text="MACD", row=3, col=1)
fig.update_yaxes(title_text="RSI", row=4, col=1)
 
fig.update(layout_xaxis_rangeslider_visible=False) #追加

fig.show()



---




